In [1]:
import os
import time
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: False


In [2]:
# Wait for process_data to finish

while 'experiment_data__20_steps__60_days.hkl' not in os.listdir('data/processed_data'):
    time.sleep(600)

#time.sleep(3600)

In [3]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [4]:
N_STEPS = 20
N_DAYS = 60
DATA = f'data/processed_data/experiment_data__{N_STEPS}_steps__{N_DAYS}_days.hkl'

In [5]:
# Store results
results = []

# Load one partition of folds
data = hkl.load(DATA)
action_remnant_input = data['action_remnant_input']
recurrent_remnant_input = data['recurrent_remnant_input']
prior_remnant_input = data['prior_remnant_input']
completion_remnant_target = data['completion_remnant_target']
problems_remnant_target = data['problems_remnant_target']
action_experiment_input = data['action_experiment_input']
recurrent_experiment_input = data['recurrent_experiment_input']
prior_experiment_input = data['prior_experiment_input']
completion_experiment_target = data['completion_experiment_target']
problems_experiment_target = data['problems_experiment_target']
experiment_sequence = data['experiment_sequence']
experiment_assignment_log_id = data['experiment_assignment_log_id']

# Clear session so models don't pile up
keras.backend.clear_session()

# Create model

# Action Model
action_input_layer = Input(shape=action_remnant_input[0].shape, name='action')
action_model = Dropout(rate=0.5)(action_input_layer)
action_model_hook = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
action_model = Dropout(rate=0.5)(action_model_hook)

action_completion_output_layer = Dense(units=1, activation='sigmoid', name='action_completion')(action_model)
action_problems_output_layer = Dense(units=1, activation='linear', name='action_problems')(action_model)

action_model = Model(action_input_layer, [action_completion_output_layer, action_problems_output_layer])
action_model.compile(optimizer='adam', loss={'action_completion': 'binary_crossentropy', 'action_problems': 'mse'})


# Recurrent Model
recurrent_input_layer = Input(shape=recurrent_remnant_input[0].shape, name='recurrent')
recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
recurrent_model = Dropout(rate=0.5)(recurrent_model)
recurrent_model_hook = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
recurrent_model = Dropout(rate=0.5)(recurrent_model_hook)

recurrent_completion_output_layer = Dense(units=1, activation='sigmoid', name='recurrent_completion')(recurrent_model)
recurrent_problems_output_layer = Dense(units=1, activation='linear', name='recurrent_problems')(recurrent_model)

recurrent_model = Model(recurrent_input_layer, [recurrent_completion_output_layer, recurrent_problems_output_layer])
recurrent_model.compile(optimizer='adam', loss={'recurrent_completion': 'binary_crossentropy', 'recurrent_problems': 'mse'})


# Prior Model
prior_input_layer = Input(shape=prior_remnant_input[0].shape, name='prior')
prior_model = Dropout(rate=0.5)(prior_input_layer)
prior_model_hook = Dense(units=64, activation='sigmoid')(prior_model)
prior_model = Dropout(rate=0.5)(prior_model_hook)

prior_completion_output_layer = Dense(units=1, activation='sigmoid', name='prior_completion')(prior_model)
prior_problems_output_layer = Dense(units=1, activation='linear', name='prior_problems')(prior_model)

prior_model = Model(prior_input_layer, [prior_completion_output_layer, prior_problems_output_layer])
prior_model.compile(optimizer='adam', loss={'prior_completion': 'binary_crossentropy', 'prior_problems': 'mse'})


# Combined Model
combined_model = Concatenate()([action_model_hook, recurrent_model_hook, prior_model_hook])
combined_model = Dropout(rate=0.5)(combined_model)
combined_completion_output_layer = Dense(units=1, activation='sigmoid', name='combined_completion')(combined_model)
combined_problems_output_layer = Dense(units=1, activation='linear', name='combined_problems')(combined_model)

combined_model = Model([action_input_layer, recurrent_input_layer, prior_input_layer], [combined_completion_output_layer, combined_problems_output_layer])
combined_model.compile(optimizer='adam', loss={'combined_completion': 'binary_crossentropy', 'combined_problems': 'mse'})


# Train Models
es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]


# Train Partial Models
weights = {'action_completion': np.ones_like(completion_remnant_target) * 16, 'action_problems': completion_remnant_target}
action_model.fit(x={'action': action_remnant_input},
                 y={'action_completion': completion_remnant_target, 'action_problems': problems_remnant_target},
                 epochs=1000,
                 validation_split=0.25,
                 callbacks=es,
                 sample_weight=weights,
                 verbose=1)

completion_experiment_output, problems_experiment_output = action_model.predict({'action': action_experiment_input})
df = pd.DataFrame(zip(np.array(['action'] * experiment_sequence.size), 
                      experiment_sequence.flatten(), 
                      experiment_assignment_log_id.flatten(), 
                      completion_experiment_target.flatten(), 
                      problems_experiment_target.flatten(), 
                      completion_experiment_output.flatten(), 
                      problems_experiment_output.flatten()), 
                  columns = ['model', 
                             'sequence_id', 
                             'assignment_log_id', 
                             'completion_target', 
                             'problems_target', 
                             'completion_prediction', 
                             'problems_prediction'])
results.append(df)


weights = {'recurrent_completion': np.ones_like(completion_remnant_target) * 16, 'recurrent_problems': completion_remnant_target}
recurrent_model.fit(x={'recurrent': recurrent_remnant_input},
                    y={'recurrent_completion': completion_remnant_target, 'recurrent_problems': problems_remnant_target},
                    epochs=1000,
                    validation_split=0.25,
                    callbacks=es,
                    sample_weight=weights,
                    verbose=1)

completion_experiment_output, problems_experiment_output = recurrent_model.predict({'recurrent': recurrent_experiment_input})
df = pd.DataFrame(zip(np.array(['assignment'] * experiment_sequence.size), 
                      experiment_sequence.flatten(), 
                      experiment_assignment_log_id.flatten(), 
                      completion_experiment_target.flatten(), 
                      problems_experiment_target.flatten(), 
                      completion_experiment_output.flatten(), 
                      problems_experiment_output.flatten()), 
                  columns = ['model', 
                             'sequence_id', 
                             'assignment_log_id', 
                             'completion_target', 
                             'problems_target', 
                             'completion_prediction', 
                             'problems_prediction'])
results.append(df)


weights = {'prior_completion': np.ones_like(completion_remnant_target) * 16, 'prior_problems': completion_remnant_target}
prior_model.fit(x={'prior': prior_remnant_input},
                y={'prior_completion': completion_remnant_target, 'prior_problems': problems_remnant_target},
                epochs=1000,
                validation_split=0.25,
                callbacks=es,
                sample_weight=weights,
                verbose=1)

completion_experiment_output, problems_experiment_output = prior_model.predict({'prior': prior_experiment_input})
df = pd.DataFrame(zip(np.array(['student'] * experiment_sequence.size), 
                      experiment_sequence.flatten(), 
                      experiment_assignment_log_id.flatten(), 
                      completion_experiment_target.flatten(), 
                      problems_experiment_target.flatten(), 
                      completion_experiment_output.flatten(), 
                      problems_experiment_output.flatten()), 
                  columns = ['model', 
                             'sequence_id', 
                             'assignment_log_id', 
                             'completion_target', 
                             'problems_target', 
                             'completion_prediction', 
                             'problems_prediction'])
results.append(df)

weights = {'combined_completion': np.ones_like(completion_remnant_target) * 16, 'combined_problems': completion_remnant_target}
combined_model.fit(x={'action': action_remnant_input, 'recurrent': recurrent_remnant_input, 'prior': prior_remnant_input},
                   y={'combined_completion': completion_remnant_target, 'combined_problems': problems_remnant_target},
                   epochs=1000,
                   validation_split=0.25,
                   callbacks=es,
                   sample_weight=weights,
                   verbose=1)

# Store model predictions
completion_experiment_output, problems_experiment_output = combined_model.predict({'action': action_experiment_input, 
                                                                                   'recurrent': recurrent_experiment_input, 
                                                                                   'prior': prior_experiment_input})
df = pd.DataFrame(zip(np.array(['combined'] * experiment_sequence.size), 
                      experiment_sequence.flatten(), 
                      experiment_assignment_log_id.flatten(), 
                      completion_experiment_target.flatten(), 
                      problems_experiment_target.flatten(), 
                      completion_experiment_output.flatten(), 
                      problems_experiment_output.flatten()), 
                  columns = ['model', 
                             'sequence_id', 
                             'assignment_log_id', 
                             'completion_target', 
                             'problems_target', 
                             'completion_prediction', 
                             'problems_prediction'])
results.append(df)

results = pd.concat(results)
experiment_conditions = pd.read_csv('data/experiment_information/experiment_conditions.csv').drop('sequence_id', axis=1)
results = results.merge(experiment_conditions, how='left', on='assignment_log_id')
results.to_csv('experiment_results.csv', index=False)

Epoch 1/1000
3578/3578 [==============================] - 215s 60ms/step - loss: 20.9316 - action_completion_loss: 9.0110 - action_problems_loss: 11.9207 - val_loss: 14.9495 - val_action_completion_loss: 9.8909 - val_action_problems_loss: 5.0586
Epoch 2/1000
3578/3578 [==============================] - 216s 60ms/step - loss: 16.1518 - action_completion_loss: 8.8343 - action_problems_loss: 7.3175 - val_loss: 14.8118 - val_action_completion_loss: 9.7281 - val_action_problems_loss: 5.0837
Epoch 3/1000
3578/3578 [==============================] - 210s 59ms/step - loss: 15.9250 - action_completion_loss: 8.7594 - action_problems_loss: 7.1656 - val_loss: 14.6078 - val_action_completion_loss: 9.6139 - val_action_problems_loss: 4.9938
Epoch 4/1000
3578/3578 [==============================] - 212s 59ms/step - loss: 15.7820 - action_completion_loss: 8.7038 - action_problems_loss: 7.0782 - val_loss: 14.5300 - val_action_completion_loss: 9.5068 - val_action_problems_loss: 5.0232
Epoch 5/1000
3578/3

3578/3578 [==============================] - 248s 69ms/step - loss: 14.4340 - combined_completion_loss: 7.6018 - combined_problems_loss: 6.8322 - val_loss: 13.3158 - val_combined_completion_loss: 8.4626 - val_combined_problems_loss: 4.8532
Epoch 5/1000
3578/3578 [==============================] - 248s 69ms/step - loss: 14.4207 - combined_completion_loss: 7.6001 - combined_problems_loss: 6.8206 - val_loss: 13.2854 - val_combined_completion_loss: 8.4403 - val_combined_problems_loss: 4.8451
Epoch 6/1000
3578/3578 [==============================] - 241s 67ms/step - loss: 14.3794 - combined_completion_loss: 7.5711 - combined_problems_loss: 6.8084 - val_loss: 13.3307 - val_combined_completion_loss: 8.4744 - val_combined_problems_loss: 4.8563
Epoch 7/1000
3578/3578 [==============================] - 244s 68ms/step - loss: 14.3547 - combined_completion_loss: 7.5777 - combined_problems_loss: 6.7769 - val_loss: 13.3010 - val_combined_completion_loss: 8.4500 - val_combined_problems_loss: 4.8511
E

In [6]:
# Load the data

results = pd.read_csv('experiment_results.csv')

In [7]:
# Evaluate the results

metrics = []

# Calculate the metrics for each sequence

for keys, df in results.groupby(['model', 'sequence_id']):
    
    model, sequence = keys
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([model, 
                    sequence, 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])


# Calculate the metrics for everything combined

for model, df in results.groupby('model'):

    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)

    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([model,
                    'all_data', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['model',
                                'group', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('experiment_metrics.csv', index=False)

In [8]:
metrics.iloc[-10:]

,model,group,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
350,student,PSAXP7W,64,0.666667,0.921875,0.068034,0.112896,0.079181,3.472874,-0.327498,-0.755138,0.027607,26.527422
351,student,PSAXTEE,47,0.764815,0.659574,0.047152,0.156737,0.232928,2.486858,0.070241,-0.073571,0.019975,8.918442
352,student,PSAYCFH,1903,0.771336,0.627956,0.100232,0.181707,0.223600,5.694859,-0.497558,-2.761556,0.024656,43.784396
353,student,PSAZ2G4,185,0.826179,0.572973,0.051630,0.209007,0.236254,5.961732,-0.530502,-3.945087,0.060542,43.878124
354,student,PSAZ5HX,800,0.703821,0.560000,-0.019553,0.115909,0.254530,11.570453,-0.047744,-0.331979,0.003526,391.401488
355,student,PSAZGQM,288,0.736304,0.875000,-0.006688,0.066830,0.102132,3.625835,-0.328367,-0.896371,0.012042,26.458080
356,action,all_data,101780,0.652257,0.694911,0.051181,0.056781,0.201958,3.698238,-0.207515,-0.415249,0.009925,31.608818
357,assignment,all_data,101780,0.727226,0.724946,0.125130,0.130488,0.186217,3.707255,-0.211424,-0.407008,0.019357,31.424764
358,combined,all_data,101780,0.758698,0.744380,0.175750,0.183537,0.175443,3.760128,-0.235159,-0.421152,0.038306,31.740665
359,student,all_data,101780,0.754174,0.730212,0.154286,0.168547,0.180012,3.757767,-0.230035,-0.429836,0.028052,31.934620


In [9]:
results.iloc[-10:]

,model,sequence_id,assignment_log_id,completion_target,problems_target,completion_prediction,problems_prediction,user_id,in_control,in_treatment
407110,combined,PSAUTWU,18397915,0,10,0.788966,4.171197,1129695.0,0.0,1.0
407111,combined,PSA2KKZ,18395857,1,12,0.800838,5.788163,1129849.0,0.0,1.0
407112,combined,PSAUTWT,18414481,1,5,0.857710,4.217018,1130045.0,1.0,0.0
407113,combined,PSAUUKY,18414970,0,7,0.787538,4.484480,1130054.0,1.0,0.0
407114,combined,PSAUUKY,18415013,0,7,0.295553,5.164132,1130059.0,1.0,0.0
407115,combined,PSAUUKY,18414291,1,9,0.874131,3.898782,1130061.0,1.0,0.0
407116,combined,PSAUUKY,18418421,0,7,0.791765,4.418406,1130196.0,1.0,0.0
407117,combined,PSAUTWT,18398639,0,3,0.701583,3.770533,1130203.0,0.0,1.0
407118,combined,PSAUTWT,18398571,0,2,0.527321,4.167850,1130204.0,0.0,0.0
407119,combined,PSAUTWT,18398567,0,4,0.697395,3.731581,1130219.0,0.0,1.0
